 IMPORTING THE LIBRARIES

In [ ]:
!pip install tensorflow

In [6]:
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()
import time
import difflib
import numpy
import json
import pickle
import random
import os
import tflearn

AttributeError: module 'tensorflow' has no attribute 'contrib'

PRE-PROCESSING THE DATA

In [3]:
file_path = 'intents.json'

with open(file_path) as file:
    data = json.load(file)



try:
    with open('data.pickle','rb') as f:
        words, labels, training, output = pickle.load(f)
except:
    words,labels,docs_patt,docs_tag = [],[],[],[]
    
# TOKENISATION & STEMMING
    for intent in data['intents']:
        for pattern in intent['patterns']:
            wrds = nltk.word_tokenize(pattern)
            for item in wrds:
                words.extend(wrds)
                docs_patt.append(wrds)
                docs_tag.append(intent['tag'])
                if intent['tag'] not in labels:
                    labels.append(intent['tag'])
    
    words = [stemmer.stem(w.lower()) for w in words]
    words = sorted(list(set(words)))
    labels = sorted(labels)
    
    training = []
    output = []
    
    out_empty = [0 for _ in range(len(labels))]

# BAG OF WORDS - FEATURE ENGINEERING
    
    for x,doc in enumerate(docs_patt):
        bag = []
        wrds = [stemmer.stem(w.lower()) for w in doc]
        for w in words:
            if w in wrds:
                bag.append(1)
                
            else:
                bag.append(0)
                
        output_row = out_empty[:]
        output_row[labels.index(docs_tag[x])] = 1
        
        training.append(bag)
        output.append(output_row)
        
    training = numpy.array(training)
    output = numpy.array(output)
    
    with open('data.pickle','wb') as f:
        pickle.dump((words,labels,training,output),f)       
    

MODEL BUILDING

In [ ]:
from tensorflow.python.framework import ops
ops.reset_default_graph()

net = tflearn.input_data(shape=[None,len(training[0])])
net = tflearn.fully_connected(net,8)
net = tflearn.fully_connected(net,8)
net = tflearn.fully_connected(net,len(output[0]),activation='softmax')
net = tflearn.regression(net)

model = tflearn.DNN(net)
    

BATCH GRADIENT DESCEND -> BATCH SIZE=8, NO. OF EPOCHES = 1000

In [ ]:
try:
    model.load('model.tflearn')
except:
    model = tflearn.DNN(net)
    history = model.fit(training,output,n_epoch=1000,batch_size=8,show_metric=True)
    model.save('model.tflearn')
    

INPUT PRE PROCESSING

In [ ]:
def bag_of_words(s,words):
    bag = [0 for _ in range(len(words))]
    
    s_words = nltk.word_tokenize(s)
    s_words = [stemmer.stem(word.lower()) for word in s_words]
    
    for se in s_words:
        for i,w in enumerate(words):
            if w == se:
                bag[i] = 1
    return numpy.array(bag)


In [ ]:
def words_to_list(s):
    a = []
    ns = ""
    s = s + " "
    
    for i in range(len(s)):
        if s[i] == " ":
            a.append(ns)
            ns = ""
        else:
            ns = ns + s[i]
            
    a = list(set(a))
    return a


In [ ]:
# PASSS THE FILE IN THIS FUNCTION TO CREATE A DICITIONARY OF UNIQUE VOCABULARY

def json_to_dictionary(data):
    dictionary = []
    fil_dict = []
    vocalubary = []
    for i in data['intents']:
        for pattern in i['patterns']:
            vocalubary.append(pattern.lower())
            
    for i in vocalubary:
        dictionary.append(words_to_list(i))
        
    for i in range(len(dictionary)):
        for word in dictionary[i]:
            fil_dict.append(word)
    return list(set(fil_dict))

# THIS FUNCTION CHECKS THE SPELLING IN THE SENTENCE
chatbot_vocalubary = json_to_dictionary(data)

In [ ]:
def word_checker(s):
    correct_string = ""
    for word in s.casefold().split():
        if word not in chatbot_vocalubary:
            suggestion = difflib.get_close_matches(word,chatbot_vocalubary)
            
            for x in suggestion:
                pass
            if len(suggestion) == 0:
                pass
            else:
                correct_string = correct_string + " " + str(suggestion[0])
                
        else:
            correct_string = correct_string + " " + str(word)
            
    return correct_string


CHAT FUNCTION

In [ ]:
def chat():
    print("BOT : Hi! I am your personal bot. I am here to answer queries on DBATU")
    while True:
        inp = input('YOU : ')
        if inp.lower() == 'quit' or inp == None:
            break
            
        inp_x = word_checker(inp)
        results = model.predict([bag_of_words(inp_x,words)])[0]
        results_index = numpy.argmax(results)
        tag = labels[results_index]
        
        if results[results_index] >= 0.9:
            for tg in data['intents']:
                if tg['tag'] == tag:
                    responses = tg['responses']
                    ms = random.choice(responses)
                    print('BOT : {}'.format(ms))
                    
                
        else:
            print("BOT : Sorry, I don't know how to answer that yet")
            
            
chat()